In [0]:
# Visualizing the performance of vanilla DQN vs temp reg DQN on games

In [2]:
!git clone https://github.com/higgsfield/RL-Adventure

Cloning into 'RL-Adventure'...
remote: Enumerating objects: 55, done.
remote: Total 55 (delta 0), reused 0 (delta 0), pack-reused 55
Unpacking objects: 100% (55/55), done.


In [3]:
!pip install numpngw

In [0]:
import math, random
import numpngw

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F

In [0]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

<h3>Use Cuda</h3>

In [0]:
USE_CUDA = torch.cuda.is_available()
Variable = lambda *args, **kwargs: autograd.Variable(*args, **kwargs).cuda() if USE_CUDA else autograd.Variable(*args, **kwargs)

<h2>Environment</h2>

In [0]:
env_id = "Acrobot-v1"
env = gym.make(env_id)

<h2>Deep Q Network</h2>

In [0]:
class DQN(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQN, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(env.observation_space.shape[0], 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, env.action_space.n)
        )
        
    def forward(self, x):
        return self.layers(x)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(state).unsqueeze(0), volatile=True)
            q_value = self.forward(state)
            action = int(q_value.max(1)[1].data[0].cpu().int().numpy())
        else:
            action = random.randrange(env.action_space.n)
        return action

In [0]:
model = DQN(env.observation_space.shape[0], env.action_space.n)

if USE_CUDA:
    model = model.cuda()
    
#optimizer = optim.Adam(model.parameters())

In [0]:
def plot(frame_idx, rewards, losses):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, np.mean(rewards[-10:])))
    plt.plot(rewards)
    plt.subplot(132)
    plt.title('loss')
    plt.plot(losses)
    plt.show()

<h2>Testing the trained policies</h2>

In [28]:
#loading weights
for noise_level_dup in [0,10,20,30,40,60,80]:
    print("_"*50)
    for noise_level in [0,10,20,30,40,60,80]:
        avg = 0.
        for i in range(1,4):
            PATH = env_id+',noise-level='+str(noise_level)+',weights_run='+str(i)+'.pt'
            model.load_state_dict(torch.load(PATH))
            model.eval()

            all_rewards = []
            action_changes = 0
            num_test_seeds = 10
            start_test_seed = 4

            for seed in range(start_test_seed,start_test_seed+num_test_seeds):
                #setting the seed
                i=seed
                torch.backends.cudnn.benchmark = False
                torch.backends.cudnn.deterministic = True
                random.seed(i)
                np.random.seed(i)
                env.seed(i)
                torch.manual_seed(i)
                torch.cuda.manual_seed_all(i)

                num_episodes = 10

                for eps in range(num_episodes):
                    episode_reward = 0
                    done = False
                    state = env.reset()
                    prev_action = -100
                    action_changes-=1
                    while done!=True:
                        epsilon = 0.0  #setting epsilon during testing
                        #for ind in range(len(env.observation_space.high)):
                        #    std = (abs(env.observation_space.high[ind]-env.observation_space.low[ind]))/noise_level_dup
                        #    if std>50000:
                        #        std = 50/noise_level_dup
                        #    std = std/2
                        #    x = np.random.normal(loc=0.0,scale=std)
                        #    if state[ind]+x>=env.observation_space.low[ind] and state[ind]+x<=env.observation_space.high[ind]:
                        #        state[ind] += x
                        action = model.act(state, epsilon)
                        if action!=prev_action:
                            action_changes+=1
                        prev_action = action
                        next_state, reward, done, _ = env.step(action)  
                        state = next_state
                        episode_reward += reward
                    all_rewards.append(episode_reward)

            avg += np.mean(np.array(all_rewards))
            #print(all_rewards)
            #print("Avg. episodic reward = "+str(np.mean(np.array(all_rewards))))
            #print("Avg. number of action changes over consecutive timesteps = "+str(0.+action_changes/(num_test_seeds*num_episodes)))
        print(avg/3)

__________________________________________________


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


-169.89666666666668
-500.0
-426.8
-377.62000000000006
-382.6433333333334
-362.1033333333333
-366.03999999999996
__________________________________________________


KeyboardInterrupt: ignored

<p><hr></p>

<h1>Atari Environment</h1>

In [11]:
%cd RL-Adventure/
!rm common/__init__.py

/content/RL-Adventure


In [12]:
# Enabling frame skip to be a parameter
%%writefile common/wrappers.py
import numpy as np
from collections import deque
import gym
from gym import spaces
import cv2
cv2.ocl.setUseOpenCL(False)

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def reset(self):
        return self.env.reset()

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)

class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255,
            shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]

class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[0], shp[1], shp[2] * k), dtype=np.uint8)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=2)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

def make_atari(env_id,frame_skip=4):
    env = gym.make(env_id)
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=frame_skip)
    return env

def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env



class ImageToPyTorch(gym.ObservationWrapper):
    """
    Image shape to num_channels x weight x height
    """
    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0.0, high=1.0, shape=(old_shape[-1], old_shape[0], old_shape[1]), dtype=np.uint8)

    def observation(self, observation):
        return np.swapaxes(observation, 2, 0)
    

def wrap_pytorch(env):
    return ImageToPyTorch(env)

Overwriting common/wrappers.py


In [0]:
from common.wrappers import make_atari, wrap_deepmind, wrap_pytorch

In [0]:
env_id = "PongNoFrameskip-v4"                    # Need to use NoFrameskip-v4 versions of Atari games in Gym, else frames will be skipped too much
env    = make_atari(env_id, frame_skip=4)        # Added frameskip parameter 
env    = wrap_deepmind(env)                      # Need to pass frame_stack=True as an argument here. 4 frames are stacked as state
env    = wrap_pytorch(env)

In [0]:
class CnnDQN(nn.Module):
    def __init__(self, input_shape, num_actions):
        super(CnnDQN, self).__init__()
        
        self.input_shape = input_shape
        self.num_actions = num_actions
        
        self.features = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU()
        )
        
        self.fc = nn.Sequential(
            nn.Linear(self.feature_size(), 512),
            nn.ReLU(),
            nn.Linear(512, self.num_actions)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
    
    def feature_size(self):
        return self.features(autograd.Variable(torch.zeros(1, *self.input_shape))).view(1, -1).size(1)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = Variable(torch.FloatTensor(np.float32(state)).unsqueeze(0), volatile=True)
            q_value = self.forward(state)
            action = int(q_value.max(1)[1].data[0].cpu().int().numpy())
        else:
            action = random.randrange(env.action_space.n)
        return action

In [0]:
model = CnnDQN(env.observation_space.shape, env.action_space.n)

if USE_CUDA:
    model = model.cuda()
    
#optimizer = optim.Adam(model.parameters(), lr=0.00001)

In [17]:
%cd ..

/content


In [20]:
!ls

'PongNoFrameskip-v4,noise=baseline,weights_run=1.pt'
'PongNoFrameskip-v4,noise=color1,weights_run=1.pt'
'PongNoFrameskip-v4,noise=color2,weights_run=1.pt'
'PongNoFrameskip-v4,noise=noise10,weights_run=1.pt'
'PongNoFrameskip-v4,noise=noise5,weights_run=1.pt'
'PongNoFrameskip-v4,noise=white1,weights_run=1.pt'
'PongNoFrameskip-v4,noise=white2,weights_run=1.pt'
 RL-Adventure
 sample_data


In [30]:
#loading weights
for noise_type in ["baseline","white1","white2"]:
    PATH = "PongNoFrameskip-v4,noise="+noise_type+",weights_run=1.pt"
    model.load_state_dict(torch.load(PATH))
    model.eval()

    all_rewards = []
    action_changes = 0
    num_test_seeds = 10
    start_test_seed = 4

    for seed in range(start_test_seed,start_test_seed+num_test_seeds):
        #setting the seed
        print(seed)
        i=seed
        torch.backends.cudnn.benchmark = False
        torch.backends.cudnn.deterministic = True
        random.seed(i)
        np.random.seed(i)
        env.seed(i)
        torch.manual_seed(i)
        torch.cuda.manual_seed_all(i)

        num_episodes = 3

        for eps in range(num_episodes):
            episode_reward = 0
            done = False
            state = env.reset()
            prev_action = -100
            action_changes-=1
            #gif_list = [state[0]]
            while done!=True:
                epsilon = 0.0  #setting epsilon during testing
                #state[0] = (state[0]+np.random.randint(256))%256
                #state[0] = state[0]+np.random.randint(low=0,high=256,size=(state[0].shape[0],state[0].shape[1]))*((np.random.randint(100,size=(state[0].shape[0],state[0].shape[1]))<1).astype(int))
                state[0] = state[0]+np.random.normal(loc=0,scale=2,size=(state[0].shape[0],state[0].shape[1]))*((np.random.randint(100,size=(state[0].shape[0],state[0].shape[1]))<100).astype(int))
                state[0] = state[0]%256
                action = model.act(state, epsilon)
                if action!=prev_action:
                    action_changes+=1
                prev_action = action
                next_state, reward, done, _ = env.step(action)  
                state = next_state
                #gif_list.append(state[0])
                episode_reward += reward
            all_rewards.append(episode_reward)
            #numpngw.write_apng(PATH+"_"+str(eps+1)+'.png', gif_list, delay=100, use_palette=True)

    #print(all_rewards)
    print("Avg. episodic reward = "+str(np.mean(np.array(all_rewards))))
    print("Avg. number of action changes over consecutive timesteps = "+str(0.+action_changes/(num_test_seeds*num_episodes)))

4


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


5
6
7
8
9
10
11
12
13
Avg. episodic reward = 17.7
Avg. number of action changes over consecutive timesteps = 1392.1666666666667
4
5
6
7
8
9
10
11
12
13
Avg. episodic reward = 17.533333333333335
Avg. number of action changes over consecutive timesteps = 1515.7333333333333
4
5
6
7
8
9
10
11
12
13
Avg. episodic reward = 19.966666666666665
Avg. number of action changes over consecutive timesteps = 1330.7666666666667
